In [57]:
import torch 
import torchvision
from torchvision import transforms
from torch.utils import data

def data_iter(batch_size):
    trans=transforms.ToTensor()
    train=torchvision.datasets.FashionMNIST(root="../data",transform=trans,train=True,download=True)
    test=torchvision.datasets.FashionMNIST(root="../data",transform=trans,train=False,download=True)
    return data.DataLoader(train,batch_size,True),data.DataLoader(train,batch_size,True)

batch_size=10
train_iter,test_iter=data_iter(batch_size)
for x,y in train_iter:
    print(x.shape)
    break

torch.Size([10, 1, 28, 28])


In [58]:
from torch import nn
num_inputs,num_outputs,num_hiddens=784,10,256

w1=nn.Parameter(torch.randn(num_inputs,num_hiddens,requires_grad=True)*0.01)
b1=nn.Parameter(torch.zeros(num_hiddens,requires_grad=True))
w2=nn.Parameter(torch.randn(num_hiddens,num_outputs,requires_grad=True)*0.01)
b2=nn.Parameter(torch.zeros(num_outputs,requires_grad=True))

params=[w1,b1,w2,b2]

In [59]:
def relu(x):
    a=torch.zeros_like(x)
    return torch.max(x,a)

def net(x):
    x=x.reshape(-1,num_inputs)
    h=relu(x@w1+b1)
    return h@w2+b2

loss=nn.CrossEntropyLoss(reduction="none")

num_epoch=200
lr=0.01
updater=torch.optim.SGD(params,lr)

In [60]:
class Accumulator():
    def __init__(self,n):
        self.data=[0.0]*n
    
    def add(self,*args):
        self.data=[ a+ float(b) for a,b in zip(self.data,args)]
    
    def __getitem__(self,index):
        return self.data[index]
    
def accuracy(y_hat,y):
    y_hat=y_hat.argmax(axis=1)
    cmd=y_hat.type(y.dtype)==y
    return float(cmd.type(y.dtype).sum())

for i in range(num_epoch):
    metric=Accumulator(3)
    for x,y in train_iter:
        y_hat=net(x)
        l=loss(y_hat,y)
        updater.zero_grad()
        l.mean().backward()
        updater.step()
        metric.add(l.sum(),accuracy(y_hat,y),y.numel())
    print(f'epoch:{i} loss {metric[0]}  y_hat:{metric[1]} y:{metric[2]}')

epoch:0 loss 45083.028244018555  y_hat:44152.0 y:60000.0
epoch:1 loss 28555.4699806273  y_hat:49997.0 y:60000.0
epoch:2 loss 25669.110057651997  y_hat:50929.0 y:60000.0
epoch:3 loss 23880.690123431385  y_hat:51672.0 y:60000.0
epoch:4 loss 22614.457714669406  y_hat:51995.0 y:60000.0
epoch:5 loss 21579.100431796163  y_hat:52357.0 y:60000.0
epoch:6 loss 20802.052203051746  y_hat:52613.0 y:60000.0
epoch:7 loss 19973.207052253187  y_hat:52873.0 y:60000.0
epoch:8 loss 19378.843111578375  y_hat:53055.0 y:60000.0
epoch:9 loss 18787.832694925368  y_hat:53213.0 y:60000.0
epoch:10 loss 18264.034186843783  y_hat:53404.0 y:60000.0
epoch:11 loss 17853.711026322097  y_hat:53609.0 y:60000.0
epoch:12 loss 17412.7532058917  y_hat:53671.0 y:60000.0
epoch:13 loss 16954.476578190923  y_hat:53869.0 y:60000.0
epoch:14 loss 16582.85491720587  y_hat:54021.0 y:60000.0
epoch:15 loss 16305.5009746477  y_hat:54144.0 y:60000.0
epoch:16 loss 15924.360378005542  y_hat:54206.0 y:60000.0
epoch:17 loss 15608.73435212113

In [61]:
def pred(test_iter):
    cal=0
    total=0
    labels=torchvision.datasets.FashionMNIST.classes
    print(labels)
    for x,y in test_iter:
        predin=net(x).argmax(axis=1)
        r= predin==y
        cal+=r.type(y.dtype).sum()
        total+=y.numel()
    return cal/total

print(pred(test_iter))




['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
tensor(0.9988)
